# Defining Nominal Approaches in fmdtools

Nominal simulation approaches are used to evaluate the performance of a model over a set of input parameters. It can then be used to:
- define/understand the operational envelope for different system parameters (i.e., what inputs can the system safely encounter)
- quantify failure probabilities given stochastic inputs (i.e., if the statistical distribution of inputs are known, what is the resulting probability of hazards given the design)

In [1]:
from fmdtools.define.model import Model
from fmdtools.define.block import FxnBlock

import fmdtools.analyze as an
import fmdtools.sim.propagate as prop

The rover model (rover_model.py) from the examples module will be used to demonstrate this approach. The main task of the rover is to follow a given line from a starting location to an ending location. It model is made of following functions and flows.

Functions:
    - Power
    - Operator
    - Communications
    - Perception
    - Avionics
    - Override
    - Drive
    - Environment

Flows:
    - Groud
    - Pos_Signal
    - EE_12
    - EE_15
    - EE_5
    - Video
    - Avionics_Control
    - Motor_control
    - Switch
    - Comms
    - Ovverride_comms
    - Faultystates

It has helper functions that help visualize the rovers behavior overtime.

    - plot_map(mdl, mdlhist): helps visualize the ability of the rover to track a given line
    - plot_trajectories(
                            mdlhists,
                            nomhist=[],
                            app=[],
                            faultlabel="Faulty",
                            faultalpha=0.1,
                            range_hist={},
                            rangealpha=0.1,
                            setalpha=0.3,
                            show_labels=True,
                            title="Fault Trajectories",
                            textoffset=2.0,
                            mode_trunc=5,
                            mode_trunc_end=5,
                            xlim=None,
                            ylim=None,
                            figsize=(4, 4),
                            ax=False,
                            legend=True,
                        ): helps visualize the rover trajectories

    - plot_centerline_err(mdl, mdlhist): helps visualize how mucht the rover has deviated from the centerline.

Additionally, this model has a corresponding *parameter generation* class which generates the design parameters of the model given a reduced space of input parameters. The pssoible parameter inputs and their dafaults are listed below.

    linetype: str = "sine"              # line type (sine or turn)
    period: float = 1.0                 # period of the curve (for sine linetype)
    end: tuple = (10.0, 10.0)           # end of the curve (requires instantiation)
    initangle: float = 0.0              # initial rover angle
    linetype_set = ("sine", "turn")
    amp: float = 1.0                    # amplitude of sine wave   (input for sine linetype)
    wavelength: float = 50.0            # wavelength of sine wave (input for sine linetype)
    radius: float = 20.0                # radius of turn (input for turn linetype)
    start: float = 20.0                 # start of turn (input for turn linetype)
    ub_f: float = 10.0                  
    lb_f: float = -1.0
    ub_t: float = 10.0
    lb_t: float = -1.0
    ub_d: float = 2.0
    lb_d: float = -2.0
    cor_d: float = 1.0
    cor_t: float = 1.0
    cor_f: float = 1.0
    degradation: DegParam = DegParam()  # Degradation parameters
    drive_modes: dict = {"mode_args": "set"}   

In [2]:
from rover_model import Rover, plot_map, plot_centerline_err, RoverParam

ImportError: cannot import name 'plot_map' from 'rover_model' (C:\Users\dhulse\Documents\GitHub\fmdtools\examples\rover\rover_model.py)

Below shows the performance of the rover during a (default) turn with a radius of 20 meters that begins at 20 meters. As shown, there is a slight drift from the centerline, but not enough for the rover to get lost (that would take 1 meter of drift).

In [3]:
p = RoverParam(linetype="turn")
mdl = Rover(p=p)
results, mdlhist = prop.nominal(mdl)
plot_map(mdl, mdlhist)

Below shows the performance of the model over a sine wave. As shown, similar to the turn line type, the drift small enough to where the rover completes its mission within acceptable bounds.

In [4]:
p = RoverParam(linetype="sine")
mdl = Rover(p=p)
results, mdlhist = prop.nominal(mdl)
plot_map(mdl, mdlhist)

The performance of the rover in these situations is dependent on the parameters of the situation (e.g., the radius of the curve and the amplitude of the sine wave). Thus, it is important to define the operational envelope for the system. This can be done using a `NominalApproach`, which can be used to define ranges of variables to simulate the system under.

In [5]:
from fmdtools.sim.approach import NominalApproach

In [6]:
help(NominalApproach)

In this approach we define parameter ranges for the two major situations--a wavelength and amplitude for the sine wave, and a radius and start location for the turn.

Defining an approach in terms of ranges is performed with `.add_param_ranges()`

In [7]:
nomapp = NominalApproach()
help(nomapp.add_param_ranges)

In [8]:
nomapp.add_param_ranges(RoverParam,'sine', linetype='sine', amp=(0, 10, 0.2), wavelength=(10.0,50.0,10.0))
nomapp.add_param_ranges(RoverParam,'turn', linetype='turn', radius=(5.0,40.0,5.0), start=(0.0, 20.0,5.0))

Notice that `RoverParam` is the class that generatse the nominal parameters of the model, `linetype = 'sine'` is a fixed parameter defining a discrete case of scenarios, and `amp` and `wavelenth` are parameters that are varied (the given tuples define the respective ranges).

The result is a defined set of scenarios which can be run in the model:

In [9]:
nomapp.scenarios

Nominal Approaches are simulated using `prop.nominal_approach`.

In [10]:
help(prop.nominal_approach)

In [11]:
endclasses, mdlhists= prop.nominal_approach(mdl, nomapp)

To speed up execution over large numbers of scenarios, multiprocessing can also be used to run the scenarios in parallel by passing an execution pool. This is not done here because it would require the model to be in a different file, and because the gains on a light-weight model like this are not significant.

Now that the approach has been simulated, the operational envelope can be visualized. There are three methods to perform this visualization `an.plot.nominal_vals_1d`, `an.plot.nominal_vals_2d`, and `an.plot.nominal_vals_3d`, which each plot the *classification* of the model in the 1/2/3 dimensions over the set of given parameters as nominal or incomplete. 

Note that this classification must be in the dictionary returned from the Model's `find_classification` function at the end of the model run under the key `classification` as is done in the rover model. This classification must also be encoded as a string.

In [12]:
help(an.plot.nominal_vals_2d)

We can then use these results to visualize the operational envelope for the system over each case. In this case, the parameter ranges of the sine wave are plotted, showing that the rover can only a low ration of amplitude to wavelenght.

In [13]:
fig = an.plot.nominal_vals_2d(nomapp, endclasses, 'inputparams.amp', 'inputparams.wavelength', metric = 'at_finish')

The plot below shows the same results for the turn parameters.

In [14]:
fig = an.plot.nominal_vals_2d(nomapp, endclasses, 'inputparams.radius', 'inputparams.start', metric = 'at_finish')

Because the primary effect is one in terms of radius, we might visualize this trend in one dimension instead:

In [15]:
fig = an.plot.nominal_vals_1d(nomapp, endclasses, 'inputparams.radius', metric = 'at_finish')

While this is helpful for plotting string classifications, we also might want to compare numeric quantities (e.g., costs, hazard probabilities, etc) over the set of factors. For this, `an.tabulate.nominal_factor_comparison` is used, which creates a table of  metrics over a given set of parameters.

In [16]:
help(an.tabulate.nominal_factor_comparison)

In [17]:
endclasses.state_probabilities()

In [18]:
nomtab = an.tabulate.nominal_factor_comparison(nomapp, endclasses, ['inputparams.radius', 'inputparams.start'], rangeid='turn', percent=False)

In [ ]:
nomtab

This table can also be summarized on individual factors:

In [ ]:
nomtab_summ = an.tabulate.nominal_factor_comparison(nomapp, endclasses, ['start'], rangeid='turn', percent=False)
nomtab_summ

`an.plot.nominal_factor_comparison` can then be used to visualize one metric from this table as a bar plot.

In [ ]:
help(an.plot.nominal_factor_comparison)

In [ ]:
fig = an.plot.nominal_factor_comparison(nomtab_summ, 'cost', ylabel='cost', title='average cost at different start locations', maxy=150)

## Quantifying probabilities

Given the ability to simulate over ranges, it can additionally be used to quantify probabilities of the different end-state classifications. `an.process.state_probabilities(endclasses)` can be used to quantify the probability these classifications.

The default probability model over ranges is to assume a uniform distribution and only assume one range has been added. This can lead to fallacious results:

In [19]:
state_probabilities = endclasses.state_probabilities()
state_probabilities

Thus, `.assoc_probs` to:
- associate the probabilities with their corresponding distributions (which may be non-uniform), and
- rebalance the overall probability of discrete cases
when running the approach over ranges. 

In [20]:
help(nomapp.assoc_probs)

Here, each case is given a weight defining the probability of the discrete case, while the corresponding parameters are given corresponding pdf functions (in this case uniform distributions from the scipy stats package).

In [21]:
from scipy import stats

In [22]:
nomapp.assoc_probs('sine', prob_weight=0.5, amp=(stats.uniform.pdf, {'loc':0,'scale':10}), wavelength=(stats.uniform.pdf,{'loc':10, 'scale':40}))
nomapp.assoc_probs('turn', prob_weight=0.5, start=(stats.uniform.pdf, {'loc':5,'scale':10}), radius=(stats.uniform.pdf,{'loc':5, 'scale':30}))

Now, when the scenarios are run, they should have the correct corresponding probabilities:

In [23]:
endclasses, mdlhists= prop.nominal_approach(mdl, nomapp)

In [24]:
state_probabilities = endclasses.state_probabilities()
state_probabilities

This result (while seemingly extreme due to the large ranges explored) is consistent with the operational envelopes presented earlier.

## Random input generation

It may additionally be helpful to perform simulations over randomly generated inputs, when there are many parameters and it is difficult to easily quantify the stochastic process. This approach is called Monte Carlo sampling, and enables one to quantify probabilities given stochasticly-generated inputs. Random inputs are assigned using `.add_rand_params`

In [25]:
nomapp_rand = NominalApproach()
help(nomapp_rand.add_rand_params)

Below, the same probability model/scenarios used above will be generated using stochastic inputs (rather than a range).

In [ ]:
import numpy as np

In [ ]:
nomapp_rand.add_rand_params(RoverParam,'sine', 'sine', prob_weight=0.5, replicates=100, amp=(np.random.uniform, 0, 10), wavelength=(np.random.uniform,10,40))
nomapp_rand.add_rand_params(RoverParam,'turn', 'turn', prob_weight=0.5, replicates=100, radius=(np.random.uniform,5,40), start=(np.random.uniform,0, 20))

We can go through the same process to verify that it tracks the range/pdf method.

In [ ]:
endclasses, mdlhists= prop.nominal_approach(mdl, nomapp_rand)

In [ ]:
fig = an.plot.nominal_vals_2d(nomapp_rand, endclasses,'inputparams.amp', 'inputparams.wavelength', metric = 'at_finish')

In [ ]:
fig = an.plot.nominal_vals_2d(nomapp_rand, endclasses, 'inputparams.radius', 'inputparams.start', metric = 'at_finish')

In [ ]:
fig = an.plot.nominal_vals_1d(nomapp_rand, endclasses, 'inputparams.radius', metric = 'at_finish')

As shown, these results track the uniform approach, although the spread catches some non-nominal scenarios that were not caught using uniform sampling. The resulting probabilities are:

In [ ]:
state_probabilities = an.process.state_probabilities(endclasses)
state_probabilities

Which is similar to (but not exactly the same as) the uniform approach. The error of Monte Carlo sampling approaches can be reduced by increasing the number of points, but these points increase computational costs.

## Nested Scenario Sampling

Thus far, we have introduced two types of approaches: 
- SampleApproach, which is used to evaluate the system resilience to a set of faults
- NominalApproach, which is used to evaluate system performance over a set of parameters

These both have their limitations when used alone. Simulating a `SampleApproach` using `propagate.approach` solely evaluates evaluates fault-driven hazards in a single nominal set of parameters (which may not generalize) while simulating a `NominalApproach` using `propagate.nominal_approach` evaluates the systerm performance/resilience to external parameters (But not faults).

To resolve these limitations, one can use a *nested* scenario sampling approach where a `SampleApproach` is simulated at each parameter level of a `NominalApproach`, giving the resilience of the system to faults over a set of operational parameters. This is called using the `propagate.nested_approach` method.

In [ ]:
help(prop.nested_approach)

Here we use the nominal approach generated earlier with a default sampling approach to quantify resilience.

In [ ]:
nested_endclasses, nested_mdlhists, nested_apps = prop.nested_approach(mdl, nomapp)

 The resulting endclass/mdlhist dictionary is in turn nested within operational scenarios.

In [ ]:
nested_endclasses.keys()

In [ ]:
nested_endclasses.sine_1.nominal

We can compare resilience to faults over the operational envelope using `an.tabulate.resilience_factor_comparison`, which generates at table similar to `an.tabulate.nominal_factor_comparison` except for a single metric with rows for each fault scenario.

In [ ]:
restab = an.tabulate.nested_factor_comparison(nomapp, nested_endclasses,['start'], 'cost', rangeid='turn', percent=False, difference=False)

In [ ]:
restab

These factors can further be visualized using `an.plot.nested_factor_comparison`, which generates a set of bar graphs similar to `an.plot.nominal_factor_comparison`.

In [ ]:
help(an.plot.nested_factor_comparison)

In [ ]:
fig = an.plot.nested_factor_comparison(restab, stat='cost', figsize = (6,4), xlabel='curve start point')

As shown, on average the costs are higher over fault scenarios than in their nominal counterparts. While this difference appears to be uniform for Avionics faults (`no_con`, which merely adds a fault at the times instantiated (since the rover is already in `drive` mode)), resulting in a uniform cost of 100), it changes for the Drive fault depending on the location of the curve. This is because in some cases this fault stops the rover at the finish line (when the line is short), and sometimes during its mission (when the line is long), leading to a cost from the incomplete mission.

Thus, the assessed consequences of faults are somewhat prone to parameters leading to a different injection time. We might prefer, for example, for avionics faults to be injected when they would make a difference (i.e. at the start) and for drive faults to be injected multiple. This can be fixed by:
- encoding phase information in with modes to ensure they are injected at the right intervals
- using the `get_phases` option in `propagate.nested_approach` to get the phases and inject the faults in the corresponding times
- passing appropriate Approach arguments to `propagate.nested_approach` (e.g., defaultsamp, etc)

This has already been done in the Power faults, which we evaluate below. (note that they were not included above because they were listed to be injected in specific phases using the `key_phases_by` option in `assoc_modes()`).

In [ ]:
nested_endclasses_power, nested_mdlhists_power, nested_apps_power = prop.nested_approach(mdl, nomapp, get_phases=True, faults='Power')

In [ ]:
restab_power = an.tabulate.nested_factor_comparison(nomapp, nested_endclasses_power,['start'], 'cost', rangeid='turn', difference=True, percent=False, faults='modes')
restab_power

In [ ]:
fig = an.plot.nested_factor_comparison(restab_power, stat='cost', figsize = (6,4), xlabel='curve start point', stack=True, maxy=800)

As shown, all power faults result in the same cost, since they all result in both a fault and an incomplete mission, in part because none of them are injected after the rover completes its mission (which would happen if we were using the global phases as in the drive faults).